# Earthquake Awareness With LLM

This Jupyter Notebook demonstrates a project built using the [ReAct Framework](https://arxiv.org/abs/2210.03629) in combination with the OpenAI API, ArcGIS World Geocoding Service, and USGS real-time earthquake feeds. 

By integrating GPT-4 with external APIs, this project bridges the fields of Large Language Models (LLMs), Geographic Information Systems (GIS), and natural hazard risk analysis. It showcases the potential for interdisciplinary research and real-world applications, unlocking new opportunities for impactful solutions.

## 1. Imports

In [1]:
# Langchain imports
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import BaseChatPromptTemplate
from langchain.schema import AgentAction, AgentFinish, HumanMessage
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI

# Standard library imports
from typing import List, Union
import requests
import json
import re


## 2. Tool definition

This section defines two tools that leverage external APIs to provide geospatial and earthquake-related functionalities:

- **ArcGIS World Geocoding Service**: Converts a place name into geographic coordinates (latitude and longitude). This tool enables seamless geocoding for GIS workflows.
- **USGS Earthquake Real-Time Feeds**: Retrieves the count of earthquakes within a specified time window and geographic region. Parameters include location, radius, and minimum magnitude, facilitating advanced earthquake data analysis.

In [2]:
# --- Tool: Geocode using ArcGIS REST API ---
def geocode(placename):
    url = "https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/findAddressCandidates"
    params = {
        "SingleLine": placename,
        "f": "json"
    }
    response = requests.get(url, params=params)
    candidates = response.json().get("candidates", [])
    if candidates:
        location = candidates[0]["location"]
        return location["y"], location["x"]  # latitude, longitude
    else:
        raise ValueError(f"Could not geocode location: {placename}")

# --- Tool: Earthquake Count ---
def get_earthquake_count(starttime, endtime, latitude, longitude, maxradius, minmagnitude):
    url = "https://earthquake.usgs.gov/fdsnws/event/1/count"
    params = {
        "starttime": starttime,
        "endtime": endtime,
        "latitude": latitude,
        "longitude": longitude,
        "maxradius": maxradius,
        "minmagnitude": minmagnitude,
        "format": "geojson"
    }
    response = requests.get(url, params=params)
    return response.json().get("count", 0)

## 3. ReAct Implementation with Langchain

This section demonstrates a ReAct (Reasoning + Acting) implementation using the Langchain framework. The ReAct framework enables the integration of reasoning and action-taking capabilities, allowing the agent to interact with tools, process intermediate steps, and arrive at a final answer in a structured manner.

The Langchain framework provides a robust and modular approach to building applications powered by Large Language Models (LLMs). By leveraging Langchain, developers can seamlessly integrate external tools, define custom prompts, and implement advanced workflows like ReAct. This enhances the agent's ability to perform complex tasks, such as geospatial analysis and earthquake data retrieval, while maintaining flexibility and scalability in the application design.

In [3]:
# --- LangChain Tool Wrappers ---
geocode_tool = Tool(
    name="Geocode",
    func=lambda place: str(geocode(place)),
    description="Returns (latitude, longitude) for a place using ArcGIS REST API. Input should be a place name."
)

earthquake_tool = Tool(
    name="EarthquakeCount",
    func=lambda args: str(
        get_earthquake_count(
            starttime=json.loads(args).get("starttime"),
            endtime=json.loads(args).get("endtime"),
            latitude=json.loads(args).get("latitude"),
            longitude=json.loads(args).get("longitude"),
            maxradius=json.loads(args).get("maxradius", 1),
            minmagnitude=json.loads(args).get("minmagnitude", 1)
        )
    ),
    description="Returns number of earthquakes in a time window and region. Input should be a dict with keys: starttime, endtime, latitude, longitude, maxradius (in degree), minmagnitude."
)

In [4]:
# --- Custom Prompt Template ---
class CustomPromptTemplate(BaseChatPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]
    
    def format_messages(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
            
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        formatted = self.template.format(**kwargs)
        return [HumanMessage(content=formatted)]
    
# --- Custom Output Parser ---
class CustomOutputParser(AgentOutputParser):
    
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        
        # Parse out the action and action input
        regex = r"Action: (.*?)[\n]*Action Input:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        
        # If it can't parse the output it raises an error
        # You can add your own logic here to handle errors in a different way i.e. pass to a human, give a canned response
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

# --- Prompt Template Setup ---
tools = [geocode_tool, earthquake_tool]
template = """
You are a GIS professional with expertise in natural hazards and disaster risk analysis.
Answer the following question using available tools.
You have access to the following tools:

{tools}

Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
{agent_scratchpad}"""

prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    input_variables=["input", "intermediate_steps"]
)

In [5]:
# Initiate our LLM (Language Model) - using 'gpt-4.1' with a temperature of 0 for deterministic responses
llm = ChatOpenAI(model='gpt-4.1', temperature=0)

# Create an LLM chain that combines the LLM and the custom prompt template
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Extract tool names from the list of tools for use in the agent
tool_names = [tool.name for tool in tools]

# Create a custom output parser to handle the LLM's output
output_parser = CustomOutputParser()

# Create an agent that uses the LLM chain, output parser, and tools
agent = LLMSingleActionAgent(
    llm_chain=llm_chain,  # The LLM chain to process input and generate output
    output_parser=output_parser,  # The parser to interpret the LLM's output
    stop=["\nObservation:"],  # Stop sequence to end processing after tool output
    allowed_tools=tool_names  # List of tools the agent is allowed to use
)


/var/folders/6g/xqgvnv4s1h95kfz0bd14h3_h0000gn/T/ipykernel_17286/1619865486.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model='gpt-4.1', temperature=0)
/var/folders/6g/xqgvnv4s1h95kfz0bd14h3_h0000gn/T/ipykernel_17286/1619865486.py:5: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=llm, prompt=prompt)
/var/folders/6g/xqgvnv4s1h95kfz0bd14h3_h0000gn/T/ipykernel_17286/1619865486.py:14: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with 

In [6]:
# Create an agent executor that combines the agent with the tools
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

## 4. Ask LLM

In [7]:
# Run the agent executor with a sample question
agent_executor.run("Is there any significant earthquake in Riverside, CA in Jan 2024?")

/var/folders/6g/xqgvnv4s1h95kfz0bd14h3_h0000gn/T/ipykernel_17286/2960268015.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent_executor.run("Is there any significant earthquake in Riverside, CA in Jan 2024?")




> Entering new AgentExecutor chain...
Thought: First, I need to get the coordinates (latitude and longitude) for Riverside, CA. Then, I will check for earthquakes in January 2024 near that location with a significant magnitude (let's use minmagnitude 4.0 as a threshold for "significant").
Action: Geocode
Action Input: Riverside, CA

Observation:(33.980534, -117.377025)
Now that I have the coordinates for Riverside, CA, I will check for earthquakes with a magnitude of at least 4.0 within a reasonable radius (let's use 50 km, which is approximately 0.45 degrees) during January 2024.
Action: EarthquakeCount
Action Input: {"starttime": "2024-01-01", "endtime": "2024-01-31", "latitude": 33.980534, "longitude": -117.377025, "maxradius": 0.45, "minmagnitude": 4.0}

Observation:2
I now know the final answer
Final Answer: Yes, there were 2 significant earthquakes (magnitude 4.0 or higher) in or near Riverside, CA during January 2024.

> Finished chain.


'Yes, there were 2 significant earthquakes (magnitude 4.0 or higher) in or near Riverside, CA during January 2024.'

# Reference

Jarvis, C., & Palermo, J. (2023, June 13). How to call functions with Chat models. OpenAI. https://cookbook.openai.com/examples/how_to_call_functions_with_chat_models

Jarvis, C. (2023, May 2). How to build a tool-using agent with LangChain. OpenAI. https://cookbook.openai.com/examples/how_to_build_a_tool-using_agent_with_langchain

Noun, T. (2023, March 14). Creating a ReAct agent from scratch using OpenAI: No frameworks required. Medium. https://medium.com/@original2547/creating-a-react-agent-from-scratch-using-openai-no-frameworks-required-111910f887f8

Yao, S., Zhao, J., Zhang, D., Ktitarev, A., Radev, D., & Liu, D. (2022). ReAct: Synergizing reasoning and acting in language models (arXiv preprint arXiv:2210.03629). https://arxiv.org/abs/2210.03629